<center><h1> Предсказание активности покупателей </h1></center>


### Описание:

Заказчик предоставил обезличенные данные по истории покупок трех интернет-магазинов


### Цель проекта: 
разработать решение, которое будет предсказывать, совершит ли пользователь повторную покупку в течение 30 дней

#### Исходные данные:

* marketplace - мультикатегорийный магазин с посещаемостью более 8 млн сессий в месяц

* apparel - магазин одежды, по характеристикам похожий на маркетплейс

* gifts – магазин подарков, очень маленький

#### В датасете представлены данные о покупках (купленных товарах):

* client_id – идентификатор клиента

* quantity – количество товара в заказе

* price – цена 1 единицы товара в заказе

* message_id – идентификатор сообщения из рассылки

* date – дата покупки

### Проект выполнила:
Никифорова Ксения Сергеевна, https://t.me/ksenia_n1

### Импорт необходимых библиотек

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [3]:
RANDOM_STATE = 12345

# Предварительный анализ данных

In [32]:
apparel = pd.read_csv('data/apparel-purchases.csv')
gift = pd.read_csv('data/gifts-purchases.csv')
marketplace = pd.read_csv('data/marketplace-purchases.csv')

In [33]:
apparel.head()

,client_id,quantity,price,date,message_id,recommended_by
0,1515915625468068833,1,3499.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
1,1515915625468068833,1,3499.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
2,1515915625468068833,1,3499.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
3,1515915625468068833,1,2450.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message
4,1515915625468068833,1,2450.0,2023-09-08,1515915625468068833-13781-64fad81bece56,bulk_message


In [34]:
gift.head()

,client_id,quantity,price,date,message_id,recommended_by
0,1515915625803894158,1,8943.0,2023-10-11,1515915625935652010-16166-6525589bca68f,bulk_message
1,1515915625803894158,1,46369.0,2023-10-11,1515915625935652010-16166-6525589bca68f,bulk_message
2,1515915625803894158,1,4666.0,2023-10-11,1515915625935652010-16166-6525589bca68f,bulk_message
3,1515915625803894158,1,11963.0,2023-10-11,1515915625935652010-16166-6525589bca68f,bulk_message
4,1515915625803894158,1,13107.0,2023-10-11,1515915625935652010-16166-6525589bca68f,bulk_message


In [35]:
marketplace.head()

,client_id,quantity,price,message_id,created_at,date
0,1515915625440944408,1,6298.0,1515915625440944408-1752-6450b79616277,2023-05-02,2023-05-02 08:49:57
1,1515915625440992498,1,7318.0,1515915625440992498-1818-646b35d089c98,2023-05-22,2023-05-22 13:52:13
2,1515915625441026424,1,65798.0,1515915625441026424-1802-64634e24a2cb9,2023-05-17,2023-05-17 11:21:04
3,1515915625441118180,1,5598.0,1515915625441118180-1820-646c9444ebf39,2023-05-23,2023-05-23 19:02:53
4,1515915625441124500,1,77998.0,1515915625441124500-1825-646ed5cf4d5ed,2023-05-25,2023-05-25 07:46:52


# Предобработка данных

In [36]:
apparel.duplicated().sum()

59250

In [37]:
gift.duplicated().sum()

56

In [38]:
marketplace.duplicated().sum()

2159

Наличие дубликатов говорит о том, что в заказе были товары с одинаковой стоимостью.

In [39]:
gift['recommended_by'].unique()

array(['bulk_message'], dtype=object)

In [40]:
apparel = apparel.drop(['recommended_by'], axis=1)
gift = gift.drop(['recommended_by'], axis=1)

Удалила столбец recommended_by, потому что в нем значение, одинаковое для всех строк. Пользу в обучении он не принесет.

In [41]:
apparel['profit'] = apparel['quantity'] * apparel['price']

In [42]:
apparel = apparel.drop(['quantity', 'price'], axis=1)

In [43]:
gift['profit'] = gift['quantity'] * gift['price']
gift = gift.drop(['quantity', 'price'], axis=1)

In [44]:
data = pd.concat([apparel, gift]).reset_index(drop=True)

In [45]:
marketplace['profit'] = marketplace['quantity'] * marketplace['price']
marketplace = marketplace.drop(['quantity', 'price'], axis=1)

In [46]:
marketplace = marketplace.drop(['date'], axis=1)

In [47]:
marketplace = marketplace.rename(columns={'created_at': 'date'})

In [48]:
marketplace = marketplace[['client_id', 'date', 'message_id', 'profit']]

In [49]:
data = pd.concat([data, marketplace]).reset_index(drop=True)

Создала новый столбец profit, в котором указала прибыль с одного заказа, а столбцы quantity и price удалила.

In [50]:
data_new = data.groupby(['client_id', 'date']).sum()
data_new = data_new.reset_index()

In [51]:
data_new

,client_id,date,message_id,profit
0,1515915625440099873,2023-07-16,1515915625440099873-1965-64abd07b9561d,31998.0
1,1515915625440099873,2023-08-09,1515915625440099873-2021-64d219c458813,1218.0
2,1515915625440939402,2023-01-30,1515915625440939402-1474-63d738fa8d5cc,75998.0
3,1515915625440944408,2023-03-01,1515915625440944408-1587-63fef11f97d27,882.0
4,1515915625440944408,2023-05-02,1515915625440944408-1752-6450b79616277,6298.0
...,...,...,...,...
57507,1515915626005014483,2023-09-13,1515915625947180414-15482-65006ded73d08,7534.0
57508,1515915626010039507,2023-10-02,1515915626010039507-15945-651ac5ef2b97e1515915...,26039.0
57509,1515915626011484905,2023-10-13,1515915626011484905-16222-65291f460849e1515915...,32913.0
57510,1515915626012131867,2023-09-12,1515915625851867569-15482-65006ded71e21,25834.0


In [52]:
data_new['target'] = 0

In [53]:
data_new['date'] = pd.to_datetime(data_new['date'])

In [54]:
data = data_new

In [55]:
uniq_client = data['client_id'].unique()

In [56]:
lst = []
for i in uniq_client:
    if len(data[data['client_id']==i]) > 1:
        lst.append(i)

In [57]:
for i in lst:
    client = data[data['client_id']==i]['date'].reset_index(drop=True)
    t = 0
    for j in range(len(client) - 1):
        for k in range(j+1, len(client)):
            if 0 < abs((client[j] - client[k]).days) <= 30:
                t = 1
    if t == 1:
        data.loc[data['client_id'] == i, 'target'] = 1

Сделала таргет: если следующая покупка была сделана в течение 30 дней - 1, если не было покупок - 0

In [58]:
# баланс классов
print('Дисбаланс классов')
print('таргет = 0 =>', (data['target'] == 0).sum() * 100 / 57512, '%')
print('таргет = 1 =>', (data['target'] == 1).sum() * 100 / 57512, '%')

Дисбаланс классов
таргет = 0 => 79.89636945333147 %
таргет = 1 => 20.103630546668523 %


In [59]:
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

Добавила столбцы день и месяц, а столбец даты удалила

In [60]:
data = data.drop(['date'], axis=1)

In [61]:
data['client_id'] = data['client_id'].astype('str')

In [62]:
data['num'] = ''

Удаляю первую часть message_id, так как она совпадает с client_id. Также я заметила, что после 19 символа начинается повторение символов, и я решила их удалить.

In [63]:
for i in range(len(data)):
    data.loc[i, 'message_id'] = data.loc[i, 'message_id'][20:]
for i in range(len(data)):
    if len(data.loc[i, 'message_id']) > 19:
        data.loc[i, 'message_id'] = data.loc[i, 'message_id'][:20] # так как там идет дальше айди клиента и еще повторяется

Разделяю message_id на 2 части

In [64]:
for i in range(len(data)):
    if len(data.loc[i, 'message_id']) == 18:
        data.loc[i, 'num'] = data.loc[i, 'message_id'][:4]
        data.loc[i, 'message_id'] = data.loc[i, 'message_id'][5:]
    else:
        data.loc[i, 'num'] = data.loc[i, 'message_id'][:5]
        data.loc[i, 'message_id'] = data.loc[i, 'message_id'][6:]

In [65]:
for i in range(len(data)):
    if '-' in data.loc[i, 'num']:
        data.loc[i, 'num'] = data.loc[i, 'num'].replace('-','')

In [66]:
data['num'] = data['num'].astype('int')

# Обучение моделей

In [67]:
scaler = StandardScaler()
X = data[['profit', 'month', 'day', 'num']]
y = data['target']

X = scaler.fit_transform(X)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [69]:
model =  RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced')
grid = GridSearchCV(model, param_grid={'max_depth': range(5, 10), 'n_estimators': range(50, 251, 100)}, 
                    scoring='roc_auc', cv=5)
grid.fit(X_train, y_train)
grid.best_score_
# до скейлера 0.6188936762383298

0.6602073723250838

In [70]:
%%time
params = {
    'loss_function' : 'Logloss',
    'random_state' : RANDOM_STATE
    }
model = CatBoostClassifier(params)
grid_cat = GridSearchCV(model, param_grid={'learning_rate': np.arange(0.05, 0.31, 0.05), 'depth': [6, 7, 8, 9],
                                           'iterations': [300]}, scoring='roc_auc', cv=5)
grid_cat.fit(X_train, y_train)

0:	learn: 0.6703019	total: 161ms	remaining: 48.1s
1:	learn: 0.6496642	total: 167ms	remaining: 24.9s
2:	learn: 0.6313675	total: 173ms	remaining: 17.1s
3:	learn: 0.6155069	total: 179ms	remaining: 13.2s
4:	learn: 0.6008566	total: 186ms	remaining: 10.9s
5:	learn: 0.5880056	total: 191ms	remaining: 9.38s
6:	learn: 0.5763520	total: 197ms	remaining: 8.26s
7:	learn: 0.5660617	total: 204ms	remaining: 7.45s
8:	learn: 0.5569290	total: 211ms	remaining: 6.81s
9:	learn: 0.5487368	total: 217ms	remaining: 6.29s
10:	learn: 0.5413705	total: 223ms	remaining: 5.87s
11:	learn: 0.5349802	total: 230ms	remaining: 5.53s
12:	learn: 0.5293486	total: 237ms	remaining: 5.22s
13:	learn: 0.5241433	total: 243ms	remaining: 4.97s
14:	learn: 0.5195487	total: 250ms	remaining: 4.74s
15:	learn: 0.5153302	total: 256ms	remaining: 4.54s
16:	learn: 0.5115063	total: 263ms	remaining: 4.37s
17:	learn: 0.5082680	total: 270ms	remaining: 4.22s
18:	learn: 0.5052689	total: 276ms	remaining: 4.08s
19:	learn: 0.5025867	total: 282ms	remaini

181:	learn: 0.4705908	total: 1.39s	remaining: 904ms
182:	learn: 0.4705516	total: 1.4s	remaining: 896ms
183:	learn: 0.4705091	total: 1.41s	remaining: 888ms
184:	learn: 0.4704489	total: 1.42s	remaining: 880ms
185:	learn: 0.4704038	total: 1.42s	remaining: 872ms
186:	learn: 0.4703669	total: 1.43s	remaining: 864ms
187:	learn: 0.4703115	total: 1.44s	remaining: 856ms
188:	learn: 0.4702665	total: 1.44s	remaining: 848ms
189:	learn: 0.4702394	total: 1.45s	remaining: 840ms
190:	learn: 0.4702113	total: 1.46s	remaining: 831ms
191:	learn: 0.4701605	total: 1.46s	remaining: 823ms
192:	learn: 0.4701148	total: 1.47s	remaining: 815ms
193:	learn: 0.4700911	total: 1.48s	remaining: 807ms
194:	learn: 0.4700236	total: 1.48s	remaining: 799ms
195:	learn: 0.4699826	total: 1.49s	remaining: 791ms
196:	learn: 0.4699375	total: 1.5s	remaining: 783ms
197:	learn: 0.4698879	total: 1.5s	remaining: 775ms
198:	learn: 0.4698337	total: 1.51s	remaining: 767ms
199:	learn: 0.4697840	total: 1.52s	remaining: 758ms
200:	learn: 0.4

61:	learn: 0.4776505	total: 417ms	remaining: 1.6s
62:	learn: 0.4774615	total: 424ms	remaining: 1.59s
63:	learn: 0.4773614	total: 432ms	remaining: 1.59s
64:	learn: 0.4772597	total: 440ms	remaining: 1.59s
65:	learn: 0.4771622	total: 447ms	remaining: 1.58s
66:	learn: 0.4770989	total: 454ms	remaining: 1.58s
67:	learn: 0.4770137	total: 461ms	remaining: 1.57s
68:	learn: 0.4769636	total: 468ms	remaining: 1.56s
69:	learn: 0.4768664	total: 474ms	remaining: 1.56s
70:	learn: 0.4767760	total: 481ms	remaining: 1.55s
71:	learn: 0.4767122	total: 488ms	remaining: 1.54s
72:	learn: 0.4766391	total: 494ms	remaining: 1.53s
73:	learn: 0.4765655	total: 500ms	remaining: 1.53s
74:	learn: 0.4764803	total: 507ms	remaining: 1.52s
75:	learn: 0.4764183	total: 514ms	remaining: 1.51s
76:	learn: 0.4763859	total: 520ms	remaining: 1.5s
77:	learn: 0.4763567	total: 527ms	remaining: 1.5s
78:	learn: 0.4762791	total: 533ms	remaining: 1.49s
79:	learn: 0.4762307	total: 541ms	remaining: 1.49s
80:	learn: 0.4761142	total: 548ms	

238:	learn: 0.4690398	total: 1.65s	remaining: 421ms
239:	learn: 0.4689837	total: 1.66s	remaining: 415ms
240:	learn: 0.4689345	total: 1.67s	remaining: 409ms
241:	learn: 0.4689045	total: 1.68s	remaining: 402ms
242:	learn: 0.4688427	total: 1.68s	remaining: 395ms
243:	learn: 0.4687931	total: 1.69s	remaining: 388ms
244:	learn: 0.4687586	total: 1.7s	remaining: 382ms
245:	learn: 0.4687068	total: 1.71s	remaining: 375ms
246:	learn: 0.4686482	total: 1.72s	remaining: 368ms
247:	learn: 0.4686009	total: 1.72s	remaining: 361ms
248:	learn: 0.4685601	total: 1.73s	remaining: 354ms
249:	learn: 0.4685082	total: 1.74s	remaining: 347ms
250:	learn: 0.4684449	total: 1.74s	remaining: 340ms
251:	learn: 0.4683922	total: 1.75s	remaining: 333ms
252:	learn: 0.4683547	total: 1.76s	remaining: 326ms
253:	learn: 0.4682875	total: 1.76s	remaining: 319ms
254:	learn: 0.4682215	total: 1.77s	remaining: 312ms
255:	learn: 0.4681483	total: 1.78s	remaining: 305ms
256:	learn: 0.4680904	total: 1.78s	remaining: 299ms
257:	learn: 0

105:	learn: 0.4749314	total: 714ms	remaining: 1.31s
106:	learn: 0.4748674	total: 722ms	remaining: 1.3s
107:	learn: 0.4748145	total: 731ms	remaining: 1.3s
108:	learn: 0.4747632	total: 737ms	remaining: 1.29s
109:	learn: 0.4747184	total: 746ms	remaining: 1.29s
110:	learn: 0.4746646	total: 753ms	remaining: 1.28s
111:	learn: 0.4746194	total: 760ms	remaining: 1.28s
112:	learn: 0.4745587	total: 767ms	remaining: 1.27s
113:	learn: 0.4744965	total: 774ms	remaining: 1.26s
114:	learn: 0.4744568	total: 781ms	remaining: 1.25s
115:	learn: 0.4744081	total: 788ms	remaining: 1.25s
116:	learn: 0.4743700	total: 795ms	remaining: 1.24s
117:	learn: 0.4743108	total: 801ms	remaining: 1.24s
118:	learn: 0.4742571	total: 809ms	remaining: 1.23s
119:	learn: 0.4742105	total: 816ms	remaining: 1.22s
120:	learn: 0.4741626	total: 823ms	remaining: 1.22s
121:	learn: 0.4741285	total: 830ms	remaining: 1.21s
122:	learn: 0.4740800	total: 837ms	remaining: 1.2s
123:	learn: 0.4740329	total: 844ms	remaining: 1.2s
124:	learn: 0.47

...

288:	learn: 0.3690748	total: 4.54s	remaining: 173ms
289:	learn: 0.3689230	total: 4.56s	remaining: 157ms
290:	learn: 0.3686448	total: 4.58s	remaining: 142ms
291:	learn: 0.3684251	total: 4.59s	remaining: 126ms
292:	learn: 0.3681689	total: 4.61s	remaining: 110ms
293:	learn: 0.3678548	total: 4.62s	remaining: 94.4ms
294:	learn: 0.3674614	total: 4.64s	remaining: 78.6ms
295:	learn: 0.3672551	total: 4.65s	remaining: 62.9ms
296:	learn: 0.3669151	total: 4.67s	remaining: 47.2ms
297:	learn: 0.3666868	total: 4.68s	remaining: 31.4ms
298:	learn: 0.3663183	total: 4.7s	remaining: 15.7ms
299:	learn: 0.3659886	total: 4.71s	remaining: 0us
0:	learn: 0.5751025	total: 16.8ms	remaining: 5.02s
1:	learn: 0.5224567	total: 27.2ms	remaining: 4.05s
2:	learn: 0.4983626	total: 47.6ms	remaining: 4.71s
3:	learn: 0.4870399	total: 66.7ms	remaining: 4.94s
4:	learn: 0.4812182	total: 87.6ms	remaining: 5.17s
5:	learn: 0.4784713	total: 106ms	remaining: 5.21s
6:	learn: 0.4762408	total: 123ms	remaining: 5.16s
7:	learn: 0.474830

154:	learn: 0.4086891	total: 2.59s	remaining: 2.42s
155:	learn: 0.4084020	total: 2.61s	remaining: 2.41s
156:	learn: 0.4080342	total: 2.62s	remaining: 2.39s
157:	learn: 0.4076645	total: 2.64s	remaining: 2.37s
158:	learn: 0.4074184	total: 2.65s	remaining: 2.35s
159:	learn: 0.4070637	total: 2.67s	remaining: 2.34s
160:	learn: 0.4066302	total: 2.69s	remaining: 2.32s
161:	learn: 0.4062642	total: 2.7s	remaining: 2.3s
162:	learn: 0.4059679	total: 2.72s	remaining: 2.28s
163:	learn: 0.4056581	total: 2.73s	remaining: 2.27s
164:	learn: 0.4053742	total: 2.75s	remaining: 2.25s
165:	learn: 0.4051620	total: 2.76s	remaining: 2.23s
166:	learn: 0.4048982	total: 2.78s	remaining: 2.21s
167:	learn: 0.4046336	total: 2.79s	remaining: 2.2s
168:	learn: 0.4043621	total: 2.81s	remaining: 2.18s
169:	learn: 0.4039664	total: 2.83s	remaining: 2.16s
170:	learn: 0.4036704	total: 2.84s	remaining: 2.15s
171:	learn: 0.4033603	total: 2.86s	remaining: 2.13s
172:	learn: 0.4030994	total: 2.87s	remaining: 2.11s
173:	learn: 0.4

27:	learn: 0.4783361	total: 260ms	remaining: 2.52s
28:	learn: 0.4781240	total: 269ms	remaining: 2.51s
29:	learn: 0.4779123	total: 281ms	remaining: 2.53s
30:	learn: 0.4777179	total: 291ms	remaining: 2.52s
31:	learn: 0.4775656	total: 301ms	remaining: 2.52s
32:	learn: 0.4773670	total: 310ms	remaining: 2.5s
33:	learn: 0.4772442	total: 320ms	remaining: 2.5s
34:	learn: 0.4771153	total: 328ms	remaining: 2.48s
35:	learn: 0.4769884	total: 338ms	remaining: 2.48s
36:	learn: 0.4768509	total: 348ms	remaining: 2.48s
37:	learn: 0.4767200	total: 357ms	remaining: 2.46s
38:	learn: 0.4765747	total: 368ms	remaining: 2.46s
39:	learn: 0.4764622	total: 376ms	remaining: 2.44s
40:	learn: 0.4763255	total: 387ms	remaining: 2.44s
41:	learn: 0.4761804	total: 395ms	remaining: 2.43s
42:	learn: 0.4760691	total: 406ms	remaining: 2.42s
43:	learn: 0.4759701	total: 417ms	remaining: 2.43s
44:	learn: 0.4758434	total: 427ms	remaining: 2.42s
45:	learn: 0.4757357	total: 436ms	remaining: 2.4s
46:	learn: 0.4756101	total: 445ms	

195:	learn: 0.4633166	total: 1.91s	remaining: 1.01s
196:	learn: 0.4632110	total: 1.93s	remaining: 1.01s
197:	learn: 0.4631376	total: 1.94s	remaining: 997ms
198:	learn: 0.4630454	total: 1.95s	remaining: 988ms
199:	learn: 0.4629802	total: 1.96s	remaining: 979ms
200:	learn: 0.4628913	total: 1.97s	remaining: 969ms
201:	learn: 0.4628146	total: 1.98s	remaining: 959ms
202:	learn: 0.4627339	total: 1.99s	remaining: 949ms
203:	learn: 0.4626292	total: 2s	remaining: 939ms
204:	learn: 0.4625372	total: 2.01s	remaining: 930ms
205:	learn: 0.4624657	total: 2.01s	remaining: 919ms
206:	learn: 0.4623924	total: 2.02s	remaining: 910ms
207:	learn: 0.4623340	total: 2.03s	remaining: 899ms
208:	learn: 0.4622578	total: 2.04s	remaining: 890ms
209:	learn: 0.4621758	total: 2.05s	remaining: 879ms
210:	learn: 0.4621095	total: 2.06s	remaining: 870ms
211:	learn: 0.4619973	total: 2.07s	remaining: 859ms
212:	learn: 0.4619444	total: 2.08s	remaining: 850ms
213:	learn: 0.4618989	total: 2.09s	remaining: 840ms
214:	learn: 0.4

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x00000235F56D4430>,
             param_grid={'depth': [6, 7, 8, 9], 'iterations': [300],
                         'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 ])},
             scoring='roc_auc')

In [71]:
grid_cat.best_score_
# 0.62 без

0.661407118180356

In [72]:
model =  LogisticRegression(random_state=RANDOM_STATE)
grid_ridge = GridSearchCV(model, param_grid={}, 
                    scoring='roc_auc', cv=5)
grid_ridge.fit(X_train, y_train)
grid_ridge.best_score_

0.6348979060039537

# Тестирование модели

In [73]:
model = CatBoostClassifier(loss_function='Logloss', learning_rate=grid_cat.best_params_['learning_rate'],
                           depth=grid_cat.best_params_['depth'],
                          iterations=grid_cat.best_params_['iterations'], 
                           random_state=RANDOM_STATE)
model.fit(X_train, y_train)
prediction_cat = model.predict(X_test)

0:	learn: 0.6483256	total: 10.8ms	remaining: 3.23s
1:	learn: 0.6127105	total: 18.8ms	remaining: 2.8s
2:	learn: 0.5850514	total: 25.9ms	remaining: 2.56s
3:	learn: 0.5636001	total: 33.8ms	remaining: 2.5s
4:	learn: 0.5462980	total: 42.4ms	remaining: 2.5s
5:	learn: 0.5327866	total: 49.4ms	remaining: 2.42s
6:	learn: 0.5225760	total: 57.2ms	remaining: 2.4s
7:	learn: 0.5137679	total: 65.5ms	remaining: 2.39s
8:	learn: 0.5070322	total: 74.9ms	remaining: 2.42s
9:	learn: 0.5016631	total: 82.3ms	remaining: 2.39s
10:	learn: 0.4973790	total: 92.2ms	remaining: 2.42s
11:	learn: 0.4938138	total: 103ms	remaining: 2.46s
12:	learn: 0.4911940	total: 110ms	remaining: 2.42s
13:	learn: 0.4887149	total: 120ms	remaining: 2.45s
14:	learn: 0.4870931	total: 127ms	remaining: 2.42s
15:	learn: 0.4853425	total: 138ms	remaining: 2.44s
16:	learn: 0.4839135	total: 145ms	remaining: 2.42s
17:	learn: 0.4828553	total: 154ms	remaining: 2.41s
18:	learn: 0.4819536	total: 164ms	remaining: 2.42s
19:	learn: 0.4812874	total: 174ms	

172:	learn: 0.4649395	total: 1.64s	remaining: 1.2s
173:	learn: 0.4648610	total: 1.65s	remaining: 1.2s
174:	learn: 0.4647862	total: 1.66s	remaining: 1.19s
175:	learn: 0.4647229	total: 1.67s	remaining: 1.18s
176:	learn: 0.4646388	total: 1.68s	remaining: 1.17s
177:	learn: 0.4645290	total: 1.69s	remaining: 1.16s
178:	learn: 0.4644492	total: 1.7s	remaining: 1.15s
179:	learn: 0.4643918	total: 1.71s	remaining: 1.14s
180:	learn: 0.4643142	total: 1.72s	remaining: 1.13s
181:	learn: 0.4642246	total: 1.73s	remaining: 1.12s
182:	learn: 0.4641746	total: 1.74s	remaining: 1.11s
183:	learn: 0.4640985	total: 1.75s	remaining: 1.1s
184:	learn: 0.4639923	total: 1.76s	remaining: 1.09s
185:	learn: 0.4639255	total: 1.77s	remaining: 1.08s
186:	learn: 0.4638551	total: 1.77s	remaining: 1.07s
187:	learn: 0.4637874	total: 1.78s	remaining: 1.06s
188:	learn: 0.4636930	total: 1.79s	remaining: 1.05s
189:	learn: 0.4636523	total: 1.8s	remaining: 1.04s
190:	learn: 0.4635816	total: 1.81s	remaining: 1.03s
191:	learn: 0.463

In [74]:
roc_auc_score(prediction_cat, y_test)

0.6131218293329295

# Выводы:

Мною была проведена предобработка данных, а также исследование с целью выбрать наилучшую модель. Модель леса и catboost показывали примерно одинаковые результаты, а логистическая регрессия показала хуже результат, поэтому я решила выбрать модель catboost.

Я разработала решение, которое предсказывает с точностью roc_auc = 0.61, совершит ли пользователь повторную покупку в течение 30 дней.